# Requirements
- fmriprep singularity image called fmriprep_v21.0.0.simg in CATD-ReliabilityAnalysis/singularity_images
- freesurfer licesnse called license.txt in CATD-ReliabilityAnalysis/singularity_images

In [1]:
from pathlib import Path
import shutil
from bids import BIDSLayout
import pandas as pd
import numpy as np
from io import StringIO
import re
import nibabel as nb
import json
import copy
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

/tmp/ipykernel_12097/417483069.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
project_root = Path('..')
bids_dir = project_root / 'data/bids_defaced'
derivatives_dir = project_root / 'data/derivatives'
swarm_cmd_dir = project_root / 'swarm/fmriprep/swarm_cmds'
sing_img_dir = Path('../singularity_images')
image_path = (sing_img_dir/'fmriprep_v21.0.0.simg').as_posix()
fs_licence_path = sing_img_dir/'license.txt'
assert fs_licence_path.exists()
fs_licence_path = fs_licence_path.as_posix()

# change this for other fmriprep runs
#run_name = 'rn_aroma'
#fmriprep_out = derivatives_dir / 'fmriprep' / run_name

#cmd_file = swarm_cmd_dir / run_name
#swarm_log_dir = project_root / ('swarm/fmriprep/swarm_logs' + run_name)
jobids = {}

In [3]:
database_path='/data/MBDU/mid_share/notebooks/pybids131'
%time layout = BIDSLayout(bids_dir, database_path=database_path, ignore=[], force_index=[])

CPU times: user 18.2 ms, sys: 4.47 ms, total: 22.6 ms
Wall time: 228 ms


/data/MBDU/mid_share/fitlins_env/lib/python3.9/site-packages/bids/layout/validation.py:46: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


In [5]:
subjects = layout.get(return_type='id', target='subject', suffix='T1w')

In [6]:
def jobhist(jid):
    jh = !jobhist {jid}
    
    jt = []
    nblanks=0
    for ll in jh:
        if nblanks >=2 and ll != '':
            jt.append(ll)
        if ll == '':
            nblanks+=1
    jhdf = pd.read_csv(StringIO('\n'.join(jt)), delim_whitespace=True)
    states = ["RUNNING", "COMPLETED", "FAILED", "CANCELLED", "TIMEOUT"]
    print("Job statuses:")
    for st in states:
        stdf = jhdf.query('State == @st')
        print(f"{st}: {len(stdf)}")
    return jhdf


def subj_status(jhdf, subjects, statuses):
    if len(jhdf) != len(subjects):
        raise ValueError('jhdf and subjects should have the same length')
    status_runs = np.array(jhdf.loc[jhdf.State.isin(statuses), 'Jobid'].str.split('_').str[1]).astype(int)
    status_subjs = np.array(subjects)[status_runs]
    return status_subjs

def put_back(orig_run_name, run_name, project_root, worked):
    bids_dir = project_root / 'data/bids'
    derivatives_dir = project_root / 'data/derivatives'
    orig_fmriprep_out = derivatives_dir / 'fmriprep' / orig_run_name
    rr_fmriprep_out = derivatives_dir / 'fmriprep' / run_name
    orig_fails = derivatives_dir / 'fmriprep' / (orig_run_name + '_failure_archive')
    assert orig_fails != rr_fmriprep_out
    orig_fails.mkdir(exist_ok=True)
    sing_cmds = []
    for sid in worked:
        rr_out_dir = rr_fmriprep_out / f'sub-{sid}'
        orig_out_dir = orig_fmriprep_out / f'sub-{sid}'
        orig_fail_dir = orig_fails / f'sub-{sid}'
        if orig_out_dir.exists():
            shutil.move(orig_out_dir, orig_fail_dir)
        assert ~orig_out_dir.exists()
        shutil.move(rr_out_dir, orig_out_dir)
    

In [6]:
#known_hard_subjects = ['24028', '22477']
known_hard_subjects = []

In [7]:
# # Move successfull runs back in to directory for original run
# orig_run_name = 'rn_aroma_20190910'
# run_name = 'rn_aroma_20190910_hard'

In [7]:
def get_completers(run_name, input_subjects):
    jhdf = jobhist(jobids[run_name])
    return subj_status(jhdf, input_subjects, statuses=['COMPLETED'])

def get_fails(run_name, input_subjects):
    jhdf = jobhist(jobids[run_name])
    return subj_status(jhdf, input_subjects, statuses=['FAILED', 'CANCELLED', 'TIMEOUT'])

# Run with fmripep 21.0.0

In [62]:
def make_sing_cmd(subj, image_path, bids_dir, fs_license_path, subj_out_dir,
                  longitudinal=False, aroma=True, dummy_scans=4,
                  save_work_directory=False, verbose=False, melodic_dims=None,
                  echo_idx=None, task=None, anat_only=False, bids_filter_file=None,
                  output_spaces=None, fs_subjects_dir=None, mnitobold=False, tedana=False,
                  rest_infos=None):
    if output_spaces is None:
        output_spaces = ['MNI152NLin2009cAsym:res-2', 'fsaverage']
    if tedana and (rest_infos is None):
        raise ValueError("If you want tedana commands created, you must past a list of dictionaries of resting scan info.")
    elif tedana:
        for ri in rest_infos:
            if ri['subject'] != str(subj):
                raise ValueError(f"Rest infos should match the subj input. subj={subj}, rest_info['subject'] = {ri['subject']}.")
    output_spaces = ' '.join(output_spaces)
    #    singularity run --cleanenv {image_path} {bids_dir} \
    #    singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep {image_path} {bids_dir} \

    sing_cmd = f'export TMPDIR=/lscratch/$SLURM_JOB_ID && \
    export SINGULARITY_BINDPATH="/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/fdb,/data,/lscratch" &&\
    mkdir -p $TMPDIR/out && \
    mkdir -p $TMPDIR/wrk && \
    singularity run --cleanenv {image_path} {bids_dir} \
    $TMPDIR/out participant \
    --participant_label {subj} \
    -w $TMPDIR/wrk \
    --nprocs $SLURM_CPUS_PER_TASK \
    --mem_mb $SLURM_MEM_PER_NODE \
    --fs-license-file {fs_licence_path} \
    --output-spaces {output_spaces} \
    --dummy-scans {dummy_scans} \
    --me-output-echos'
    if anat_only:
        sing_cmd += " --anat-only" 
    if bids_filter_file:
        sing_cmd += f" --bids-filter-file {bids_filter_file.as_posix()}"
    if longitudinal:
        sing_cmd += " --longitudinal"
    if aroma:
        sing_cmd += " --use-aroma --error-on-aroma-warnings"
        if melodic_dims is not None:
            sing_cmd += f" --aroma-melodic-dimensionality {melodic_dims}"
    if echo_idx is not None:
        sing_cmd += f" --echo-idx {echo_idx}"
    if task is not None:
        sing_cmd += f" --task-id {task}"
    if verbose:
        sing_cmd += " -vvv"
    sing_cmd += '; '
    sing_cmd += "FPSTATUS=$? ; " # Save the exit status of fmri prep, but allow the rsync to run so we can see the log
    statuses = ['FPSTATUS']
    
    if tedana:
        # make collect_fmriprep command
        cf_cmd =  f'singularity exec --cleanenv \
        {image_path} python /data/MBDU/mid_share/code/collect_fmriprep.py \
        -i $TMPDIR/out/fmriprep \
        -o $TMPDIR/wrk/fmriprep_wf \
        -s {subj}; \
        CFSTATUS=$? ; '
        statuses.append('CFSTATUS')
        
        # one command per resting state sequence
        td_cmds = []
        for rix, ri in enumerate(rest_infos):
            d = ' '.join([f'$TMPDIR/out/fmriprep/{pp}' for pp in ri['preproc_names']])
            e = ' '.join([str(et) for et in ri['echo_times']])
            out_dir = f'$TMPDIR/out/tedana/sub-{ri["subject"]}/ses-{ri["session"]}'
            prefix = ri['preproc_names'][0].split('/')[-1].split('_echo')[0]
            gscontrol = 'gsr mir'
            td_cmd = f'mkdir -p {out_dir}; tedana -d {d} -e {e} --prefix {prefix} --out-dir {out_dir} --gscontrol {gscontrol}; \
            TD{rix}STATUS=$? ; '
            statuses.append(f'TD{rix}STATUS')
            td_cmds.append(td_cmd)
            
        sing_cmd += cf_cmd + ' '.join(td_cmds)
    
    if not mnitobold:
        sing_cmd += f" rsync -ach $TMPDIR/out {subj_out_dir} ;"
        if save_work_directory:
            subj_work_dir = subj_out_dir / "wrk"
            sing_cmd += f" rsync -ach $TMPDIR/wrk {subj_work_dir} ; "
            sing_cmd += f" chown -R :MBDU {subj_work_dir} ; "

        sing_cmd += f" chown -R :MBDU {subj_out_dir} ; "
    else:
        sing_cmd += 'mkdir $TMPDIR/mnitobold; '
        sing_cmd += f' singularity exec --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep \
                    {image_path} /data/MBDU/midla/notebooks/code/run_mnitobold.sh '
        sing_cmd += ' $TMPDIR $TMPDIR/mnitobold '
        sing_cmd += ' /data/MBDU/midla/data/templates/tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz /data/MBDU/midla/data/templates/tpl-MNI152NLin2009cAsym_res-02_desc-carpet_dseg.nii.gz'
        sing_cmd += ' --omp-nthreads=$SLURM_CPUS_PER_TASK --mem-gb=$SLURM_MEM_PER_NODE ;'
        sing_cmd += " STATUS=$? ; " # Save the exit status of fmri prep, but allow the rsync to run so we can see the log
        sing_cmd += f' rsync -ach $TMPDIR/mnitobold/ /data/MBDU/midla/data/derivatives/mnitobold/run3/sub-{subj} ;'
        sing_cmd += f' chown -R :MBDU /data/MBDU/midla/data/derivatives/mnitobold/run3/sub-{subj} ;'
    
    status_cmds = ' ' + ' ; '.join([f'echo "{status} is ${status}"' for status in statuses]) + ' ; '
    status_cmds += ' ' + ' && '.join([f'(exit ${status})' for status in statuses])
    sing_cmd += status_cmds # exit a subshell with the saved status so that hpc correctly identifies it as a success or failure

    return sing_cmd


def write_swarm_file(run_name, project_root, image_path,
                     fs_license_path, subjects, layout=None, 
                     sing_sess=False, defaced=True,
                     update=False, bids_filter=None, multiecho_masking_test=False,
                     fs_subjects_run=None, **kwargs):
    if defaced:
        bids_dir = project_root / 'data/bids_defaced'
    elif multiecho_masking_test:
        bids_dir = project_root / 'data/multiecho_masking_test'
    else:
        bids_dir = project_root / 'data/bids'
    derivatives_dir = project_root / 'data/derivatives'
    swarm_cmd_dir = project_root / 'swarm/fmriprep/swarm_cmds'
    swarm_cmd_dir.mkdir(parents=True, exist_ok=True)
    fmriprep_out = derivatives_dir / 'fmriprep' / run_name
    if fs_subjects_run is not None:
        fs_base_dir = derivatives_dir / fs_subjects_run
    cmd_file = swarm_cmd_dir / run_name
    swarm_log_dir = project_root / 'swarm/fmriprep/swarm_logs' / run_name
    if (bids_filter is not None) and not sing_sess:
        fmriprep_out.mkdir(parents=True, exist_ok=True)
        bids_filter_file = fmriprep_out / 'bids_filter'
        bids_filter_file.write_text(json.dumps(bids_filter, indent=2))
        kwargs['bids_filter_file'] = bids_filter_file
    sing_cmds = []
    old_subjs = []
    for sid in subjects:
        if fs_subjects_run is not None:
            fs_subjects_dir = fs_base_dir / f'sub-{sid}/out/freesurfer'
        else:
            fs_subjects_dir = None
        if sing_sess:
            if layout is None:
                raise ValueError("Must pass a layout if you're running single session.")
            if bids_filter is None:
                bids_filter = {
                               't1w': {'datatype': 'anat', 'suffix': 'T1w'},
                              }
            sessions = layout.get(return_type='id', target='session', subject=sid, **bids_filter['t1w'])
            
            for sesid in sessions:
                ses_out_dir = fmriprep_out / f'sub-{sid}/ses-{sesid}'
                ses_out_dir.mkdir(parents=True, exist_ok=True)
                ses_filter = copy.deepcopy(bids_filter)
                for modality in ses_filter.keys():
                    ses_filter[modality]['session'] = sesid
                ses_filter_file = ses_out_dir / f'sub-{sid}_ses-{sesid}.filter'
                ses_filter_file.write_text(json.dumps(ses_filter, indent=2))
                kwargs['bids_filter_file'] = ses_filter_file
                sing_cmd = make_sing_cmd(sid, image_path, bids_dir, fs_licence_path,
                                         ses_out_dir,fs_subjects_dir=fs_subjects_dir,
                                         **kwargs)
                sing_cmds.append(sing_cmd)
        else:
            subj_out_dir = fmriprep_out / f'sub-{sid}'
            if update and (subj_out_dir / f'out/fmriprep/sub-{sid}.html').exists():
                print(f"Skipping {sid} because results for the subject exist.")
                old_subjs.append(sid)
            else:
                subj_out_dir.mkdir(parents=True, exist_ok=True)
                # TODO: make sure the ones we think failed, actually failed
                # get needed tedana inputs
                if kwargs.get('tedana', False):
                    rest_scans = sorted((bids_dir / f'sub-{sid}').glob('ses-*/func/*rest*echo-1*.nii.gz'))
                    rest_infos = []
                    for rs in rest_scans:
                        rest_info = layout.parse_file_entities(rs.as_posix())
                        rest_info['echos'] = []
                        rest_info['echo_times'] = []
                        rest_info['preproc_names'] = []
                        rs_parent = rs.parent
                        echos = sorted(rs_parent.glob(rs.parts[-1].replace('echo-1', 'echo-*')))
                        for echo in echos:
                            echo_info = layout.parse_file_entities(echo.as_posix())
                            rest_info['echos'].append(echo_info['echo'])
                            echo_time = json.loads(Path(echo.as_posix().replace('.nii.gz', '.json')).read_text())['EchoTime']
                            rest_info['echo_times'].append(echo_time*1000)
                            preproc_name = '/'.join(list(echo.parts[-4:-1]) + [echo.parts[-1].replace('_bold', '_space-scanner_desc-partialPreproc_bold')])
                            rest_info['preproc_names'].append(preproc_name)
                        rest_infos.append(rest_info)
                    kwargs['rest_infos'] = rest_infos
                sing_cmd = make_sing_cmd(sid, image_path, bids_dir, fs_licence_path, subj_out_dir,
                                         fs_subjects_dir=fs_subjects_dir, **kwargs)
                sing_cmds.append(sing_cmd)

    cmd_file = Path(cmd_file.as_posix())
    cmd_file.write_text('\n'.join(sing_cmds))

    tmp = cmd_file.read_text().split('\n')
    for i, tt in enumerate(tmp):
        print(tt)
        if i > 5:
            break
    if not sing_sess:
        assert len(tmp) == (len(subjects) - len(old_subjs))
    
    return cmd_file, swarm_log_dir, tmp

### Code below is messy, but reflects running fmriprep and debugging issues to get as many participants as possible run

In [63]:
#run_name = 'fmriprep_v21.0.0'
#subjects = [20900, 22228, 22293, 22686, 22698, 22892, 23495, 23549, 24065, 23924, 23867, 23783, 23674, 23550]
run_name = 'fmriprep_v21.0.0_fmapaffinefix'
#subjects = [21250, 23399] #23399 failed, testing 21250 where files are not modified to see if it works
subjects = [23399,23565,23649,23674,23772,23777,23809,23859,23919,23927,23937]

In [60]:
easy_subjects = [ss for ss in subjects if ss not in known_hard_subjects]

NameError: name 'known_hard_subjects' is not defined

In [64]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True, tedana=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v21.0.0.simg /data/MBDU/mid_share/data/bids_defaced     $TMPDIR/out participant     --participant_label 23399     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4     --me-output-echos --bids-filter-file /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/bids_filter --use-aroma --error-on-aroma-warnings -vvv; FPSTATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/sub-23399 ; chown -R :MBDU /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/sub-2

('fmriprep_v21.0.0_fmapaffinefix',
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprep_v21.0.0_fmapaffinefix'),
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_logs/fmriprep_v21.0.0_fmapaffinefix'))

In [28]:
# jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 247 -t 56 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name} --sbatch --constraint=cpu56
# jobids[run_name] = jobids[run_name][0]
# jobids[run_name]

In [65]:
subjects

[23399, 23565, 23649, 23674, 23772, 23777, 23809, 23859, 23919, 23927, 23937]

In [66]:
cmd_file

PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprep_v21.0.0_fmapaffinefix')

In [67]:
#jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 120 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
#manually modified /data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprep_v21.0.0_fmapaffinefix to take out /gs4 had some issues with that
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:100 -g 120 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'35895179'

In [83]:
#Get running failed_jobids with memory overruns
jhdf = jobhist(19602308)

failed_jobids = jhdf.loc[jhdf['State'] == 'RUNNING']
failed_jobids = failed_jobids.loc[failed_jobids['Jobid'] != '19461934_74']
failed_jobids = failed_jobids.Jobid

for ji in failed_jobids:
    !scancel {ji}

statuses=['FAILED', 'CANCELLED', 'TIMEOUT']
if len(jhdf) != len(subjects):
    raise ValueError('jhdf and subjects should have the same length')
status_runs = np.array(jhdf.loc[jhdf.State.isin(statuses), 'Jobid'].str.split('_').str[1]).astype(int)
status_subjs = np.array(subjects)[status_runs]

In [79]:
#jobhist(19602308)
#jobhist(35832400)
#jobhist(35838388)
#jobhist(35868168)
jobhist(35895179)

Job statuses:
RUNNING: 0
COMPLETED: 7
FAILED: 0
CANCELLED: 0
TIMEOUT: 4


,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist
0,35895179_0,norm,COMPLETED,1,24,2-00:00:00,08:53:12,120.0GB/node,27.4GB,cn0900
1,35895179_1,norm,COMPLETED,1,24,2-00:00:00,09:11:21,120.0GB/node,29.4GB,cn1014
2,35895179_2,norm,COMPLETED,1,24,2-00:00:00,08:00:04,120.0GB/node,27.7GB,cn3098
3,35895179_3,norm,COMPLETED,1,24,2-00:00:00,08:06:18,120.0GB/node,31.5GB,cn3150
4,35895179_4,norm,COMPLETED,1,24,2-00:00:00,08:30:20,120.0GB/node,30.8GB,cn3159
5,35895179_5,norm,TIMEOUT,1,24,2-00:00:00,2-00:00:03,120.0GB/node,35.8GB,cn3238
6,35895179_6,norm,TIMEOUT,1,24,2-00:00:00,2-00:00:03,120.0GB/node,36.1GB,cn3351
7,35895179_7,norm,TIMEOUT,1,24,2-00:00:00,2-00:00:03,120.0GB/node,33.3GB,cn3376
8,35895179_8,norm,COMPLETED,1,24,2-00:00:00,07:50:43,120.0GB/node,20.2GB,cn3382
9,35895179_9,norm,TIMEOUT,1,24,2-00:00:00,2-00:00:03,120.0GB/node,38.0GB,cn3396


In [80]:
#subjects[72],subjects[74]
subjects[5],subjects[6],subjects[7],subjects[9]

(23777, 23809, 23859, 23927)

In [83]:
run_name = 'fmriprep_v21.0.0_fmapaffinefix'
#these subjects timed out after 2 days, also disk storage
#subjects = [23399, 23565, 23649, 23674, 23772, 23777, 23809, 23859, 23919, 23927, 23937]
subjects = [23777,23809,23859,23927]
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v21.0.0.simg /data/MBDU/mid_share/data/bids_defaced     $TMPDIR/out participant     --participant_label 23777     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4     --me-output-echos --bids-filter-file /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/bids_filter --use-aroma --error-on-aroma-warnings -vvv; FPSTATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/sub-23777 ; chown -R :MBDU /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprep_v21.0.0_fmapaffinefix/sub-2

('fmriprep_v21.0.0_fmapaffinefix',
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprep_v21.0.0_fmapaffinefix'),
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_logs/fmriprep_v21.0.0_fmapaffinefix'))

In [84]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:1000 -g 200 -t 30 --module singularity,webproxy --time 96:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'36071960'

In [97]:
jobhist(36071960)

Job statuses:
RUNNING: 0
COMPLETED: 4
FAILED: 0
CANCELLED: 0
TIMEOUT: 0


,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist
0,36071960_0,norm,COMPLETED,1,30,4-00:00:00,07:53:53,200.0GB/node,36.8GB,cn3630
1,36071960_1,norm,COMPLETED,1,30,4-00:00:00,10:07:42,200.0GB/node,36.4GB,cn3631
2,36071960_2,norm,COMPLETED,1,30,4-00:00:00,14:59:16,200.0GB/node,33.9GB,cn3624
3,36071960_3,norm,COMPLETED,1,30,4-00:00:00,08:21:45,200.0GB/node,33.1GB,cn3625


In [71]:
run_name = 'fmriprepv20.2.2_fails_2'
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  [subjects[72],subjects[74]],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True)

run_name, cmd_file, swarm_log_dir

IndexError: list index out of range

In [31]:
disk_fail_subjects = ['sub-23490',
 'sub-23513',
 'sub-23544',
 'sub-23550',
 'sub-23554',
 'sub-23565',
 'sub-23574',
 'sub-23593',
 'sub-23607',
 'sub-23625',
 'sub-23641',
 'sub-23672',
 'sub-23745',
 'sub-23746',
 'sub-23767',
 'sub-23772',
 'sub-23783',
 'sub-23809',
 'sub-23810',
 'sub-23814',
 'sub-23830',
 'sub-23859',
 'sub-23867',
 'sub-23927',
 'sub-23951',
 'sub-23952',
 'sub-23962',
 'sub-23988',
 'sub-24028',
 'sub-24054',
 'sub-24059',
 'sub-24112']

In [84]:
run_name = 'fmriprepv20.2.2_lowmem'
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  subjects[32:],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v20.2.2.simg /data/MBDU/mid_share/data/bids_defaced     $TMPDIR/out participant     --participant_label 23399     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem/bids_filter --use-aroma --error-on-aroma-warnings -vvv;  STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem/sub-23399 ; chown -R :MBDU /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem/sub-23399 ; 
export TMPDIR=/lscra

('fmriprepv20.2.2_lowmem',
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprepv20.2.2_lowmem'),
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_logs/fmriprepv20.2.2_lowmem'))

In [85]:
swarm_log_dir

PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_logs/fmriprepv20.2.2_lowmem')

In [86]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 65 -t 25 --module singularity,webproxy --time 24:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'20067587'

In [87]:
run_name

'fmriprepv20.2.2_lowmem'

In [14]:
#lowmem
len(jobhist(20067587))

Job statuses:
RUNNING: 0
COMPLETED: 157
FAILED: 0
CANCELLED: 0
TIMEOUT: 13


170

In [11]:
jhdf = jobhist(20067587)
jhdf['subjects'] = subjects[32:]
jhdf.loc[jhdf.State == 'TIMEOUT']

Job statuses:
RUNNING: 0
COMPLETED: 157
FAILED: 0
CANCELLED: 0
TIMEOUT: 13


,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist,subjects
1,20067587_1,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:26,65.0GB/node,60.7GB,cn0744,23428
11,20067587_11,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:25,65.0GB/node,65.0GB,cn0331,23520
13,20067587_13,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:12,65.0GB/node,59.6GB,cn0156,23540
17,20067587_17,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:07,65.0GB/node,65.0GB,cn2007,23549
23,20067587_23,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:33,65.0GB/node,64.2GB,cn0145,23564
34,20067587_34,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:28,65.0GB/node,65.0GB,cn0096,23638
40,20067587_40,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:28,65.0GB/node,61.1GB,cn0641,23652
42,20067587_42,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:28,65.0GB/node,61.1GB,cn0775,23660
52,20067587_52,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:04,65.0GB/node,60.8GB,cn0208,23732
68,20067587_68,norm,TIMEOUT,1,26,1-00:00:00,1-00:00:26,65.0GB/node,35.0GB,cn1697,23798


In [9]:
jobids['fmriprepv20.2.2_lowmem'] = 20067587
fails = get_fails('fmriprepv20.2.2_lowmem',subjects[32:])
fails

Job statuses:
RUNNING: 0
COMPLETED: 157
FAILED: 0
CANCELLED: 0
TIMEOUT: 13


array(['23428', '23520', '23540', '23549', '23564', '23638', '23652',
       '23660', '23732', '23798', '23810', '23823', '23859'], dtype='<U5')

In [18]:
run_name = 'fmriprepv20.2.2_lowmem_fails1'
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  fails,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v20.2.2.simg /data/MBDU/mid_share/data/bids_defaced     $TMPDIR/out participant     --participant_label 23428     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem_fails1/bids_filter --use-aroma --error-on-aroma-warnings -vvv;  STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem_fails1/sub-23428 ; chown -R :MBDU /data/MBDU/mid_share/data/derivatives/fmriprep/fmriprepv20.2.2_lowmem_fails1/sub-23428 ; 

('fmriprepv20.2.2_lowmem_fails1',
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_cmds/fmriprepv20.2.2_lowmem_fails1'),
 PosixPath('/data/MBDU/mid_share/swarm/fmriprep/swarm_logs/fmriprepv20.2.2_lowmem_fails1'))

In [19]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 100 -t 35 --module singularity,webproxy --time 24:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'20203123'

In [22]:
jobhist(20203123)

Job statuses:
RUNNING: 13
COMPLETED: 0
FAILED: 0
CANCELLED: 0
TIMEOUT: 0


,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist
0,20203123_0,norm,RUNNING,1,36,1-00:00:00,03:05:19,100.0GB/node,-,cn3505
1,20203123_1,norm,RUNNING,1,36,1-00:00:00,03:05:19,100.0GB/node,-,cn1641
2,20203123_2,norm,RUNNING,1,36,1-00:00:00,03:05:19,100.0GB/node,-,cn1930
3,20203123_3,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn3449
4,20203123_4,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn1576
5,20203123_5,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn1742
6,20203123_6,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn1787
7,20203123_7,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn1801
8,20203123_8,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn1871
9,20203123_9,norm,RUNNING,1,36,1-00:00:00,02:31:50,100.0GB/node,-,cn2014


In [157]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544'

bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  ['23544'],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=False, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 23544     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544/bids_filter --longitudinal -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23

('fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544'))

In [158]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'58972033'

In [61]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_23638'

bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  ['23638'],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=True, t2scoreg=False, multiecho_masking_test=True)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/multiecho_masking_test      $TMPDIR/out participant     --participant_label 23638     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_23638/bids_filter --longitudinal --use-aroma --error-on-aroma-warnings -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmripre

('fmriprepv20.1.0_20200528_2mm_clifix_23638',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_23638'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_23638'))

In [62]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'59023751'

In [52]:
project_root

PosixPath('/data/MBDU/midla')

In [83]:
image_path

'/data/MBDU/singularity_images/fmriprep_20.0.5.simg'

In [159]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard'

bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  known_hard_subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=False, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 24028     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard/bids_filter -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard/sub-24028 ; c

('fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard'))

In [160]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'58972034'

In [161]:
jobids

{'fmriprepv20.0.5_20190325_2mm_2_hard': '58869771',
 'fmriprepv20.1.0_20200528_2mm_56cpulotsofmem_test': '58889357',
 'fmriprepv20.1.0_20200528_2mm_clifix_test': '58892193',
 'fmriprepv20.1.0_20200528_2mm_clifix': '58900755',
 'fmriprepv20.1.0_20200528_2mm_clifix_23544': '58900758',
 'fmriprepv20.1.0_20200528_2mm_clifix_hard': '58919112',
 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma': '58971492',
 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_23544': '58972033',
 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_hard': '58972034'}

In [27]:
jobids['fmriprepv20.1.0_20200528_2mm']

'58813266'

In [162]:
jhdf = jobhist(jobids['fmriprepv20.1.0_20200528_2mm_clifix'])


Job statuses:
RUNNING: 0
COMPLETED: 86
FAILED: 101
CANCELLED: 0


In [163]:
jhdf['subject'] = easy_subjects

In [37]:
jhdf.query('subject == "23613"')

,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist,subject
52,58813266_52,norm,FAILED,1,24,2-00:00:00,11:15:59,64.0GB/node,32.7GB,cn3548,23613


In [40]:
swarm_logs = Path('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_23544')

PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_23544')

In [41]:
log_paths = []
for jid in jhdf.Jobid:
    break

In [33]:
cc_pat = re.compile('cpu_count = ([0-9]*)')
task_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_(.*)_run_[0-9]_echo_[0-9]_wf')
echo_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_.*_run_[0-9]_echo_([0-9])_wf')
run_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_.*_run_([0-9])_echo_[0-9]_wf')
error_pat = re.compile('Error on "(.*)"')


In [42]:
jdf = []
for ix, ss in enumerate(easy_subjects):
    job_info ={}
    jobid = f'58900755_{ix}'
    job_info['subject'] = ss
    job_info['jobid'] = jobid
    job_info['log_path'] = f'/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix/fmriprepv20.1.0_20200528_2mm_clifix_{jobid}.o'
    log = Path(job_info['log_path']).read_text()
    job_info['cpu_count'] = int(cc_pat.search(log).groups()[0])
    
    try:
        
        job_info['errors'] = error_pat.findall(log)
        job_info['em1'] = job_info['errors'][0]
        job_info['nerrors'] = len(job_info['errors'])
        job_info['task'] = task_pat.findall(job_info['em1'])[0]
        job_info['echo'] = echo_pat.findall(job_info['em1'])[0]
        job_info['run'] = run_pat.findall(job_info['em1'])[0]

    except IndexError:
        pass
    jdf.append(job_info)

In [43]:
jdf = pd.DataFrame(jdf)

In [44]:
jdf['State'] = 'FAILED'
jdf.loc[jdf.nerrors.isnull(), 'State'] = 'COMPLETED'

In [45]:
em_df = jdf.copy()

In [46]:
em_df['error'] = em_df.em1.str.split('.').str[-1]

In [47]:
em_df.loc[em_df.error.isnull(), ['error', 'task', 'echo', 'run']] = 'NA'

In [50]:
em_df.groupby(['cpu_count', 'State', 'error', 'task', 'echo', 'run'])[['jobid']].count().rename(columns={'jobid':'n'})

n
cpu_count State     error                         task echo run    
32        COMPLETED NA                            NA   NA   NA   58
          FAILED    ica_aroma                     rest 1    1    33
                    ica_aroma_confound_extraction rest 1    1    15
56        COMPLETED NA                            NA   NA   NA   28
          FAILED    dvars                         rest 1    1     1
                    ica_aroma                     rest 1    1    26
                                                            2     1
                    ica_aroma_confound_extraction rest 1    1    24

In [74]:
aroma_failed = em_df.query('State == "FAILED"').subject.unique()
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_aromafail'
image_path = (sing_img_dir/'fmriprep_20.1.1.simg').as_posix()


In [75]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  aroma_failed[:5],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.1.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 20900     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_aromafail/bids_filter --longitudinal --use-aroma --error-on-aroma-warnings -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2m

('fmriprepv20.1.0_20200528_2mm_clifix_aromafail',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_aromafail'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_aromafail'))

In [76]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'59352656'

In [164]:
jhdf['log_path'] = jhdf.Jobid.apply(lambda x:f'/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix/fmriprepv20.1.0_20200528_2mm_clifix_{x}.o')
jhdf['cpu_count'] = jhdf.log_path.apply(lambda x: int(cc_pat.findall(Path(x).read_text())[0]))

In [91]:
jhdf.groupby(['cpu_count', 'State'])[['Jobid']].count().rename(columns={'Jobid':'n'})

n
cpu_count State        
32        FAILED      5
56        CANCELLED  45
          COMPLETED  67
          FAILED     47
72        CANCELLED   8
          COMPLETED  10
          FAILED      5

In [110]:
task_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_(.*)_run_[0-9]_echo_[0-9]_wf')
echo_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_.*_run_[0-9]_echo_([0-9])_wf')
run_pat = re.compile('func_preproc_ses_[0-9][0-9]_task_.*_run_([0-9])_echo_[0-9]_wf')

In [111]:
error_messages = []
for lp in jhdf.log_path:
    log = Path(lp).read_text()
    res = {}
    try:
        
        res['errors'] = error_pat.findall(log)
        res['em1'] = res['errors'][0]
        res['nerrors'] = len(res['errors'])
        res['task'] = task_pat.findall(res['em1'])[0]
        res['echo'] = echo_pat.findall(res['em1'])[0]
        res['run'] = run_pat.findall(res['em1'])[0]

    except IndexError:
        pass
    error_messages.append(res)

In [115]:
em_df = pd.concat([jhdf, pd.DataFrame(error_messages)], axis=1)
em_df['error'] = em_df.em1.str.split('.').str[-1]

In [126]:
em_df.loc[em_df.State == 'CANCELLED', 'State'] = 'MEMORY'
em_df.loc[em_df.error.isnull(), ['error', 'task', 'echo', 'run']] = 'NA'

In [127]:
em_df.groupby(['cpu_count', 'State', 'error', 'task', 'echo', 'run'])[['Jobid']].count().rename(columns={'Jobid':'n'})

n
cpu_count State     error                         task echo run    
32        FAILED    ica_aroma                     rest 1    1     5
56        COMPLETED NA                            NA   NA   NA   67
          FAILED    ica_aroma                     rest 1    1    30
                    ica_aroma_confound_extraction rest 1    1    16
                    smooth                        rest 1    1     1
          MEMORY    NA                            NA   NA   NA   15
                    ica_aroma                     rest 1    1    21
                    ica_aroma_confound_extraction rest 1    1     8
72        COMPLETED NA                            NA   NA   NA   10
          FAILED    ica_aroma                     rest 1    1     3
                    ica_aroma_confound_extraction rest 1    1     2
          MEMORY    NA                            NA   NA   NA    1
                    ica_aroma                     rest 1    1     6
                    ica_aroma_confound_extraction rest 1    1     1

In [128]:
em_df.query("State == 'MEMORY' & error == 'NA'")

,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist,subject,log_path,cpu_count,echo,em1,errors,nerrors,run,task,error
13,58813266_13,norm,MEMORY,1,24,2-00:00:00,21:59:13,64.0GB/node,64.0GB,cn1083,22698,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_13.o,72,NA,NaN,[],NaN,NA,NA,NA
20,58813266_20,norm,MEMORY,1,24,2-00:00:00,21:50:58,64.0GB/node,58.7GB,cn3185,23108,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_20.o,56,NA,NaN,[],NaN,NA,NA,NA
33,58813266_33,norm,MEMORY,1,24,2-00:00:00,21:08:35,64.0GB/node,64.0GB,cn3171,23519,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_33.o,56,NA,NaN,[],NaN,NA,NA,NA
34,58813266_34,norm,MEMORY,1,24,2-00:00:00,21:08:35,64.0GB/node,64.0GB,cn3281,23520,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_34.o,56,NA,NaN,[],NaN,NA,NA,NA
40,58813266_40,norm,MEMORY,1,24,2-00:00:00,21:02:58,64.0GB/node,64.0GB,cn3206,23550,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_40.o,56,NA,NaN,[],NaN,NA,NA,NA
41,58813266_41,norm,MEMORY,1,24,2-00:00:00,21:02:58,64.0GB/node,64.0GB,cn3295,23553,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_41.o,56,NA,NaN,[],NaN,NA,NA,NA
42,58813266_42,norm,MEMORY,1,24,2-00:00:00,21:02:58,64.0GB/node,57.5GB,cn3349,23554,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_42.o,56,NA,NaN,[],NaN,NA,NA,NA
44,58813266_44,norm,MEMORY,1,24,2-00:00:00,21:02:58,64.0GB/node,63.4GB,cn3481,23564,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_44.o,56,NA,NaN,[],NaN,NA,NA,NA
45,58813266_45,norm,MEMORY,1,24,2-00:00:00,21:02:58,64.0GB/node,61.1GB,cn3585,23565,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_45.o,56,NA,NaN,[],NaN,NA,NA,NA
71,58813266_71,norm,MEMORY,1,24,2-00:00:00,20:08:04,64.0GB/node,64.0GB,cn3486,23732,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_71.o,56,NA,NaN,[],NaN,NA,NA,NA


In [82]:
jhdf.query("cpu_count == 32 & State == 'FAILED'")

,Jobid,Partition,State,Nodes,CPUs,Walltime,Runtime,MemReq,MemUsed,Nodelist,subject,log_path,cpu_count
15,58813266_15,norm,FAILED,1,24,2-00:00:00,16:53:46,64.0GB/node,44.1GB,cn0156,22749,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_15.o,32
144,58813266_144,norm,FAILED,1,24,2-00:00:00,15:33:21,64.0GB/node,43.4GB,cn0220,24060,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_144.o,32
145,58813266_145,norm,FAILED,1,24,2-00:00:00,15:44:26,64.0GB/node,37.2GB,cn0362,24064,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_145.o,32
146,58813266_146,norm,FAILED,1,24,2-00:00:00,16:01:31,64.0GB/node,47.3GB,cn0738,24065,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_146.o,32
179,58813266_179,norm,FAILED,1,24,2-00:00:00,12:16:34,64.0GB/node,28.4GB,cn0207,24186,/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm/fmriprepv20.1.0_20200528_2mm_58813266_179.o,32


In [ ]:
run_name = 'fmriprepv20.1.0_20200528_2mm_'

bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  known_hard_subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

In [18]:
jhdf = jobhist(jobids['fmriprepv20.0.5_20190325_2mm'])
bad_states = ['FAILED', 'CANCELLED']
failed_runs = np.array(jhdf.loc[jhdf.State.isin(bad_states), 'Jobid'].str.split('_').str[1]).astype(int)
failed_subjs = np.array(subjects)[failed_runs]

Job statuses:
RUNNING: 1
COMPLETED: 100
FAILED: 88
CANCELLED: 0


In [85]:
run_name = 'fmriprepv20.0.5_20190325_2mm_2_hard'

In [86]:
known_hard_subjects = ['24028', '22477']

In [87]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  known_hard_subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=False, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 24028     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_hard/bids_filter --use-aroma --error-on-aroma-warnings -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_hard/sub-24028 ; chown -R :MBDU /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_hard/sub-24028 ;  (

('fmriprepv20.0.5_20190325_2mm_2_hard',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.0.5_20190325_2mm_2_hard'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.0.5_20190325_2mm_2_hard'))

In [88]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'58869771'

In [23]:
run_name = 'fmriprepv20.0.5_20190325_2mm_2_fails'
image_path = (sing_img_dir/'fmriprep_20.0.5.simg').as_posix()
failed_subjs = ['22698',
'23015',
'23495',
'23516',
'23544',
'23546',
'23564',
'23641',
'23652',
'23988']

In [24]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  failed_subjs,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_20.0.5.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 22698     -w $TMPDIR/wrk     --nthreads $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_fails/bids_filter --longitudinal --use-aroma --error-on-aroma-warnings -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_fails/sub-22698 ; chown -R :MBDU /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm_2_

('fmriprepv20.0.5_20190325_2mm_2_fails',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.0.5_20190325_2mm_2_fails'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.0.5_20190325_2mm_2_fails'))

In [25]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'58759467'

In [24]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  failed_subjs,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=True, t2scoreg=True)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_20.0.5.simg /data/MBDU/midla/data/bids      $TMPDIR/out participant     --participant_label 21111     -w $TMPDIR/wrk     --nthreads $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 fsaverage     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm/bids_filter --longitudinal --use-aroma --error-on-aroma-warnings --t2s-coreg -vvv ;     STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm/sub-21111 ; chown -R :MBDU /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.0.5_20190325_2mm/s

('fmriprepv20.0.5_20190325_2mm',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.0.5_20190325_2mm'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.0.5_20190325_2mm'))

In [25]:
jobids[run_name+'_2'] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name+'_2'] = jobids[run_name][0]
jobids[run_name+'_2']

'5'

In [28]:
jobids[run_name+'_2']

'5'

In [29]:
rerun = ['21669', '22127', '22477', '22749', '23015', '23108', '23298',
       '23303', '23516', '23528', '23557', '23611', '23717', '23718',
       '23720', '23753', '23770', '23783', '23809', '23813', '23825',
       '23834', '23843', '23852', '23857', '23886', '23890', '23895',
       '23900', '23905', '23906', '23919', '23927', '23935', '23943',
       '23957', '23969', '24028', '24047', '24052', '24059', '24060',
       '24064', '24065', '24077', '24078', '24088', '24093', '24101',
       '24110', '24111', '24116', '24123', '24137', '24138', '24156',
       '24158', '24167', '24176', '24177', '24180', '24186', '24204',
       '24226', '24264']

In [ ]:
run_name = 'fmriprepv20.0.5_20190325_2mm'
image_path = (sing_img_dir/'fmriprep_20.0.5.simg').as_posix()

bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=False,
                                                  verbose=True,
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=True, t2scoreg=True)

run_name, cmd_file, swarm_log_dir

In [26]:
jhdf = jobhist(jobids[run_name+'_2'])
failed = subj_status(jhdf, subjects, statuses=['FAILED', 'CANCELLED'])


EmptyDataError: No columns to parse from file

# Rerun fmriprep with freesurfer directory and bold output while saving work directory so I can see how to pull out the transforms

In [10]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'


In [11]:
easy_subjects = [ss for ss in subjects if ss not in known_hard_subjects]

In [11]:
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  easy_subjects[:2],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  output_spaces = ['MNI152NLin2009cAsym:res-2', 'func'],
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=False, t2scoreg=False,
                                                  fs_subjects_run='fmriprepv20.1.0_20200528_2mm_clifix_noaroma')

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids     $TMPDIR/out participant     --participant_label 20900     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 func     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output/bids_filter --longitudinal -vvv;  STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output/sub-20900

('fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'))

In [17]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'62826584'

In [14]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  easy_subjects,
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  output_spaces = ['MNI152NLin2009cAsym:res-2', 'func'],
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=False, t2scoreg=False, mnitobold=True,
                                                  fs_subjects_run='fmriprepv20.1.0_20200528_2mm_clifix_noaroma')

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids     $TMPDIR/out participant     --participant_label 20900     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 func     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output/bids_filter --longitudinal -vvv; mkdir $TMPDIR/mnitobold;  singularity exec --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/lib/pytho

('fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'))

In [15]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'64536703'

In [17]:
cmd_file

PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output')

In [44]:
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'
bids_filter = {
                "t1w": {
                    "datatype": "anat",
                    "reconstruction": "prenorm",
                    "suffix": "T1w"
                }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  [22477, 23638],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  defaced=False,
                                                  output_spaces = ['MNI152NLin2009cAsym:res-2', 'func'],
                                                  anat_only=False,
                                                  bids_filter=bids_filter,
                                                  longitudinal=True, aroma=False, t2scoreg=False, mnitobold=True,
                                                    
                                                  fs_subjects_run='fmriprepv20.1.0_20200528_2mm_clifix_noaroma')

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv --bind /data/MBDU/nielsond/fmriprep/fmriprep/:/usr/local/miniconda/lib/python3.7/site-packages/fmriprep /data/MBDU/singularity_images/fmriprep_20.1.0.simg /data/MBDU/midla/data/bids_defaced      $TMPDIR/out participant     --participant_label 22477     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 func     --dummy-scans 4 --bids-filter-file /data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output/bids_filter --longitudinal -vvv; mkdir $TMPDIR/mnitobold;  singularity exec --cleanenv --bind /data/MBDU/nielsond/fmriprep-fix-cli-parser/fmriprep/:/usr/local/miniconda/

('fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprepv20.1.0_20200528_2mm_clifix_noaroma_bold_output'))

In [45]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'63265649'

# Run with lts-rc to test mutli-echo

In [21]:
image_path = (sing_img_dir/'fmriprep_v20.2.0rc0.simg').as_posix()

run_name = 'fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output'
# bids_filter = {
#                 "t1w": {
#                     "datatype": "anat",
#                     "reconstruction": "prenorm",
#                     "suffix": "T1w"
#                 }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  ['20900', '21111', '21669', '22477', '23638'],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  defaced=True,
                                                  output_spaces = ['MNI152NLin2009cAsym:res-2', 'func'],
                                                  anat_only=False,
                                                  longitudinal=True, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v20.2.0rc0.simg /data/MBDU/midla/data/bids_defaced     $TMPDIR/out participant     --participant_label 20900     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 func     --dummy-scans 4 --longitudinal --use-aroma --error-on-aroma-warnings -vvv;  STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output/sub-20900 ; rsync -ach $TMPDIR/wrk /data/MBDU/midla/data/derivatives/fmriprep/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output/sub-20900/wrk ;  chown -R :MBDU /data/MBDU/midla/data/derivatives/fmr

('fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output'))

In [22]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'64649157'

In [19]:
easy_subjects[:3]

['20900', '21111', '21669']

In [24]:
image_path = (sing_img_dir/'fmriprep_v20.2.0rc0.simg').as_posix()

run_name = 'fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface'
# bids_filter = {
#                 "t1w": {
#                     "datatype": "anat",
#                     "reconstruction": "prenorm",
#                     "suffix": "T1w"
#                 }}
cmd_file, swarm_log_dir, _cmds = write_swarm_file(run_name,
                                                  project_root,
                                                  image_path,
                                                  fs_licence_path,
                                                  ['20900', '21111', '21669', '22477', '23638'],
                                                  layout=layout,
                                                  sing_sess=False,
                                                  save_work_directory=True,
                                                  verbose=True,
                                                  defaced=False,
                                                  output_spaces = ['MNI152NLin2009cAsym:res-2', 'func'],
                                                  anat_only=False,
                                                  longitudinal=True, aroma=True, t2scoreg=False)

run_name, cmd_file, swarm_log_dir

export TMPDIR=/lscratch/$SLURM_JOB_ID &&     export SINGULARITY_BINDPATH="/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data,/lscratch" &&    mkdir -p $TMPDIR/out &&     mkdir -p $TMPDIR/wrk &&     singularity run --cleanenv /data/MBDU/singularity_images/fmriprep_v20.2.0rc0.simg /data/MBDU/midla/data/bids     $TMPDIR/out participant     --participant_label 20900     -w $TMPDIR/wrk     --nprocs $SLURM_CPUS_PER_TASK     --mem_mb $SLURM_MEM_PER_NODE     --fs-license-file /data/MBDU/singularity_images/license.txt     --output-spaces MNI152NLin2009cAsym:res-2 func     --dummy-scans 4 --longitudinal --use-aroma --error-on-aroma-warnings -vvv;  STATUS=$? ;  rsync -ach $TMPDIR/out /data/MBDU/midla/data/derivatives/fmriprep/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface/sub-20900 ; rsync -ach $TMPDIR/wrk /data/MBDU/midla/data/derivatives/fmriprep/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface/sub-20900/wrk ;  chown -R :MBDU /data/MBDU/midla/data/deriv

('fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface',
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_cmds/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface'),
 PosixPath('/data/MBDU/midla/swarm/fmriprep/swarm_logs/fmriprep_v20.2.0rc0_2mm_clifix_noaroma_bold_output_nodeface'))

In [25]:
jobids[run_name] = ! swarm -f {cmd_file} --gres=lscratch:200 -g 64 -t 24 --module singularity,webproxy --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name}
jobids[run_name] = jobids[run_name][0]
jobids[run_name]

'64654916'